# Transformers 模型量化技术：GPTQ

![image](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/159_autogptq_transformers/thumbnail.jpg)

2022年，Frantar等人发表了论文 [GPTQ：Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323)。

这篇论文详细介绍了一种训练后量化算法，适用于所有通用的预训练 Transformer模型，同时只有微小的性能下降。

GPTQ算法需要通过对量化模型进行推理来校准模型的量化权重。详细的量化算法在[原始论文](https://arxiv.org/pdf/2210.17323.pdf)中有描述。

基于[`auto-gptq`](https://github.com/PanQiWei/AutoGPTQ.git) 开源实现库，transformers 支持使用GPTQ算法量化的模型。

## 使用 GPTQ 量化模型

为了使用 `auto-gptq` 库量化一个模型，我们需要向量化器传递一个数据集。

通常有两种方式构造数据集：
- 量化器支持的默认数据集（包括`['wikitext2','c4','c4-new','ptb','ptb-new']`）
- 一个字符串列表（这些字符串将被用作数据集）

### 使用支持的默认数据集来量化

在下面的示例中，让我们尝试使用`"c4"`数据集将模型量化为4位精度。支持的精度有`[2, 4, 6, 8]`。

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_id = "facebook/opt-2.7b"

quantization_config = GPTQConfig(
     bits=4, # 量化精度
     group_size=128,
     dataset="c4",
     desc_act=False,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
quant_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map='auto')

/root/miniconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/root/miniconda3/lib/python3.11/site-packages/huggingface_hub-0.19.4-py3.8.egg/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.


Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

### 实测GPU显存占用：量化模型（峰值超过7GB）

```shell
Sun Dec 24 15:21:35 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   53C    P0              71W /  70W |   7261MiB / 15360MiB |     97%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|```

#### 检查量化模型正确性

通过检查`线性层的属性`来确保模型已正确量化，它们应该包含`qweight`和`qzeros`属性，这些属性应该是`torch.int32`数据类型。

In [3]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('qweight',
               tensor([[ 1766754698, -1249142373,  1183631034,  ..., -2038658921,
                        -2037544795, -1956877206],
                       [ 1772710025,  1739893370, -1483310250,  ...,  2021033895,
                         -662329995,  1739241850],
                       [-1164093850, -2007414633, -1951893913,  ..., -1429760649,
                         -979785307,  1451914633],
                       ...,
                       [ 2025363833, -1412789579,  1539086474,  ...,   -56906855,
                        -1737062521, -1950767767],
                       [ 1753843096, -1432974698, -2057796218,  ..., -1161799560,
                         2042131817,  1450689949],
                       [ 1769502614,  1988467067, -1183147637,  ...,  1731558246,
                         2023258776,  1970754678]], device='cuda:0', dtype=torch.int32)),
              ('qzeros',
               tensor(

#### 使用 GPU 加载模型并生成文本

In [5]:
# tokenizer = AutoTokenizer.from_pretrained(model_id)

text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to see you're still here.
Thank you! I'm glad to be here too.


### 使用自定义数据集量化模型（灵活可扩展，前提是准备好数据）

下面演示通过传递自定义数据集来量化一个模型。

通过字符串列表来自定义一个数据集，建议样本数不少于128（样本数太少会影响模型性能）

In [6]:
from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer

model_id = "facebook/opt-2.7b"

quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=["auto-gptq is an easy-to-use model quantization library with user-friendly apis, based on GPTQ algorithm."]
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
quant_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, torch_dtype=torch.float16, device_map="auto")

Quantizing model.decoder.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

相比使用默认数据集，未经精心准备的自定义数据集会明显降低模型性能

In [7]:
text = "Merry Christmas! I'm glad to"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

Merry Christmas! I'm glad to.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.



## Homework：使用 GPTQ 量化 OPT-6.7B 模型